In [1]:
import numpy
import cvxpy as cvx
import control

## Solving the (continuous) Lyapunov equation
$$ A^TX +XA + Q = 0 $$

Find $X>0$ (positive semi-definite) given $P,Q > 0$ and $A$ is Hurwitz (globally asymptotically stable).

In [2]:
A = np.diag([-4, -5,-.01])
Q = np.diag([1,1,10])

### Solve with python control system library
(which calls `slycot`, which calls the Fortran library `slicot`)

In [3]:
Xcontrol = control.lyap( A, Q )
print(Xcontrol)

[[1.25e-01 0.00e+00 0.00e+00]
 [0.00e+00 1.00e-01 0.00e+00]
 [0.00e+00 0.00e+00 5.00e+02]]


### Solve as an LMI/SDP
Problem already setup as an SDP!

In [4]:
n = A.shape[0]
# construct an nxn positive semi-definite matrix
X = cvx.Variable( (n,n), PSD=True) 
# setup the constraint
constr = [ A.T@X + X@A + Q == np.zeros((n,n))]
# problem is a feasibility problem, objective unimportant
obj = cvx.Minimize( None )
prob = cvx.Problem( obj, constr)
#solve with an SDP solver (in this case SCS)
prob.solve()

if prob.solution.status == 'infeasible':
    raise RuntimeError("Problem is infeasible")

Xsdp = X.value
print(Xsdp)

[[1.25e-01 0.00e+00 0.00e+00]
 [0.00e+00 1.00e-01 0.00e+00]
 [0.00e+00 0.00e+00 5.00e+02]]


Another way to specify a positive semidefinite matrix

In [5]:
X = cvx.Variable( (n,n) )
constr = [ X.T == X, X>> 0] # X is PSD
constr.append( A.T@X + X@A + Q == np.zeros((n,n)) )
obj = cvx.Minimize( None )
prob = cvx.Problem( obj, constr)
prob.solve();

In [6]:
X.value

array([[1.25e-01, 0.00e+00, 0.00e+00],
       [0.00e+00, 1.00e-01, 0.00e+00],
       [0.00e+00, 0.00e+00, 5.00e+02]])

Another method of setting up the feasibility problem

$$
\text{min } t \\
\text{subject to } A^TX + XA +Q -tI \le 0
$$

The original problem is feasible iff $t<0$.

At least as specified in the book, if the problem is already feasible, giving the above formulation to a solver is not bounded.

In [7]:
X = cvx.Variable( (n,n) , PSD=True)
t = cvx.Variable()
I = np.identity(n)
constr = [ A.T@X + X@A +Q -t*I << 0, t>= -1000]
obj = cvx.Minimize( t )
prob = cvx.Problem(obj,constr)
prob.solve()
print(t.value)

-1000.0020908125807


In [8]:
prob.solution.status

'optimal'

### SDP approach is slower than the linear algebra approach
`slicot` approach:  https://github.com/KTH-AC/slicot/blob/master/src/SB03MD.f#L182

In [9]:
%%timeit
prob.solve();

9.86 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
control.lyap( A, Q )

28.7 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
